In [59]:
from glob import glob
from tqdm import tqdm
import json
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
from urllib.parse import urlparse

In [66]:
files = glob('orang-melayu/*.json') + glob('bahasa-melayu/*.json') + glob('sejarah/*.json') \
+ glob('melayu/*.json') + glob('sivik/*.json') + glob('orang-iban/*.json') + glob('bahasa-arab/*.json') \
+ glob('tanah-melayu/*.json')

In [67]:
links, not_in = [], []
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)
    links.extend([i['link'][0]['URL'] for i in data['message']['items'] if 'link' in i])
    not_in.extend([i['resource']['primary']['URL'] for i in data['message']['items'] if 'link' not in i])

urls = sorted(list(set(links) | set(not_in)))
urls = [u for u in urls if '.my' in urlparse(u).netloc]
len(urls)

100%|██████████████████████████████████████| 1273/1273 [00:03<00:00, 338.67it/s]


2530

In [68]:
urls

['http://abqarijournal.usim.edu.my/index.php/abqari/article/view/192',
 'http://abqarijournal.usim.edu.my/index.php/abqari/article/view/196',
 'http://alhikmah.my/index.php/hikmah/article/download/260/368',
 'http://alhikmah.my/index.php/hikmah/article/view/260',
 'http://apjee.usm.my/APJEE_31_2016/APJEE%2031_Art%204(47-68).pdf',
 'http://apjee.usm.my/APJEE_33_2018/APJEE%2033_Art6%20(75-93).pdf',
 'http://apjee.usm.my/APJEE_33_2018/APJEE%2033_Art7%20(95-108).pdf',
 'http://apjee.usm.my/APJEE_34_2019/apjee342019_3.pdf',
 'http://apjee.usm.my/APJEE_34_2019/apjee342019_7.pdf',
 'http://apjee.usm.my/APJEE_35_1_2020/apjee35012020_3.pdf',
 'http://cipsf.my/images/pdf/jipsf/apr2020/JIPSF112020_9-22.pdf',
 'http://cipsf.my/images/pdf/jipsf/june2021/JIPSF-312021_37-49.pdf',
 'http://cipsf.my/images/pdf/jipsf/oct2021/JIPSF-3_22021_40-51.pdf',
 'http://e-journal.uum.edu.my/index.php/jps/article/view/11316',
 'http://e-journal.uum.edu.my/index.php/jps/article/view/11342',
 'http://e-journal.uum.ed

In [73]:
!mkdir melayu-pdf
!rm melayu-pdf/*

mkdir: cannot create directory ‘melayu-pdf’: File exists


In [74]:
import os

def filter_a(soup):
    a = soup.find_all('a')
    a = [a_.get('href').replace('/article/view/', '/article/download/') for a_ in a if '/article/view/' in a_.get('href', '') or a_.get('href', '').endswith('.pdf')]
    return a

def download(filename, url):
    url = url.replace('/article/view/', '/article/download/')
    if os.path.exists(filename):
        return
    try:
        r = requests.get(url, timeout = 60)
    except:
        return
    if r.content[:4] == b'%PDF':
        with open(filename, 'wb') as fopen:
            fopen.write(r.content)
    else:
        soup = BeautifulSoup(r.content)
        filtered = filter_a(soup)
        for i in range(len(filtered)):
            filename_ = os.path.splitext(filename)[0]
            filename_ = f'{filename_}-{i}.pdf'
            try:
                r = requests.get(filtered[i], timeout = 60)
            except:
                continue
            with open(filename_, 'wb') as fopen:
                fopen.write(r.content)
        with open(filename, 'w') as fopen:
            fopen.write('a')

In [75]:
download('melayu-pdf/0.pdf', urls[-1])

In [77]:
urls[1]

'http://abqarijournal.usim.edu.my/index.php/abqari/article/view/196'

In [78]:
max_worker = 20

for i in tqdm(range(0, len(urls), max_worker)):
    urls_ = [(os.path.join('melayu-pdf', f'{i + no}.pdf'), url) for no, url in enumerate(urls[i: i + max_worker])]
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(download, url[0], url[1]): url for url in urls_}

        for future in as_completed(futures):
            future.result()

 43%|█████████████████▊                        | 54/127 [04:42<04:05,  3.36s/it]encoding error : input conversion failed due to input error, bytes 0x90 0x5E 0xE3 0x8C
encoding error : input conversion failed due to input error, bytes 0x90 0x5E 0xE3 0x8C
 70%|█████████████████████████████▍            | 89/127 [06:51<07:38, 12.07s/it]encoding error : input conversion failed due to input error, bytes 0x9D 0xC8 0xC0 0xC6
encoding error : input conversion failed due to input error, bytes 0x9D 0xC8 0xC0 0xC6
encoding error : input conversion failed due to input error, bytes 0x90 0xA2 0x20 0x95
encoding error : input conversion failed due to input error, bytes 0x90 0xA2 0x20 0x95
encoding error : input conversion failed due to input error, bytes 0x90 0xD3 0x20 0xF2
encoding error : input conversion failed due to input error, bytes 0x90 0xD3 0x20 0xF2
encoding error : input conversion failed due to input error, bytes 0x81 0xE5 0x82 0x2C
encoding error : input conversion failed due to input err